I've created a comprehensive Jupyter notebook for crowd disaster management and stampede prevention! This system is designed to predict dangerous situations before they occur using real-time data from traffic cameras, sensors, and other monitoring systems.
Key Features of the System:

🔍 Core Components:

Real-time Data Processing - Simulates integration with traffic cameras, IoT sensors, weather APIs
AI-Powered Risk Prediction - Uses Random Forest ML model to predict disaster probability
Anomaly Detection - Identifies unusual crowd patterns using Isolation Forest
Multi-zone Monitoring - Tracks different areas simultaneously
Automated Alert System - Generates actionable alerts with specific recommendations

📊 Risk Indicators Monitored:

Crowd Density - People per square meter vs historical averages
Flow Rate - How quickly crowds are moving through spaces
Traffic Congestion - Vehicle flow that might affect crowd movement
Bottleneck Detection - High density + low flow = danger zone
Environmental Factors - Weather, temperature, special events

🚨 Alert Levels:

CRITICAL - High disaster probability (>70%), immediate action required
WARNING - Anomalous patterns or concerning density ratios
MEDIUM - Elevated risk factors requiring monitoring
LOW - Normal operations

🎯 Predictive Capabilities:
The system can predict dangerous situations by analyzing:

Current crowd density compared to historical norms (2.5x = danger threshold)
Crowd movement velocity (slower = more dangerous)
Traffic flow impacts on pedestrian areas
Special events that increase crowd gathering
Weather conditions affecting crowd behavior

📈 Real-world Applications:

Shopping malls during sales events
Transportation hubs during peak hours
Sports venues and concert halls
Religious gatherings and festivals
Emergency evacuations

The notebook includes a complete simulation showing how the system would detect a high-risk scenario (high density, slow movement, special event) and automatically generate alerts with specific action recommendations like deploying crowd control personnel and opening alternative exits.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Machine Learning Libraries
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import DBSCAN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.optimizers import Adam
except ImportError:
    print("TensorFlow not installed. Some advanced features may not work.")

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Crowd Disaster Management and Stampede Prevention System
# Real-time Analysis and Prediction Notebook

print("🚨 CROWD DISASTER MANAGEMENT PREDICTION SYSTEM 🚨")
print("="*60)
print("Initializing real-time crowd analysis and stampede prevention system...")



In [ ]:
## 1. DATA SIMULATION AND COLLECTION FRAMEWORK

def generate_synthetic_crowd_data(n_samples=10000):
    """
    Generate synthetic crowd and traffic data for training and testing
    This simulates real-world data from cameras, sensors, and traffic systems
    """
    np.random.seed(42)
    
    # Time-based features
    timestamps = pd.date_range(start='2024-01-01', periods=n_samples, freq='5min')
    hours = [t.hour for t in timestamps]
    days_of_week = [t.dayofweek for t in timestamps]
    is_weekend = [1 if d >= 5 else 0 for d in days_of_week]
    
    # Location features (simulating different areas/zones)
    zones = np.random.choice(['Zone_A', 'Zone_B', 'Zone_C', 'Zone_D', 'Zone_E'], n_samples)
    
    # Crowd density features (people per square meter)
    base_density = np.random.normal(2.5, 1.0, n_samples)
    base_density = np.clip(base_density, 0, 10)
    
    # Traffic flow features (vehicles per minute)
    traffic_flow = np.random.normal(15, 5, n_samples)
    traffic_flow = np.clip(traffic_flow, 0, 50)
    
    # Environmental factors
    weather_conditions = np.random.choice(['Clear', 'Rainy', 'Foggy'], n_samples, p=[0.7, 0.2, 0.1])
    temperature = np.random.normal(25, 10, n_samples)
    
    # Event-based factors
    special_events = np.random.choice([0, 1], n_samples, p=[0.9, 0.1])  # 10% chance of special event
    
    # Calculate derived features
    crowd_velocity = np.random.normal(1.2, 0.3, n_samples)  # m/s
    crowd_velocity = np.clip(crowd_velocity, 0, 3)
    
    # Crowd flow rate (people crossing per minute)
    flow_rate = base_density * crowd_velocity * 10 + np.random.normal(0, 2, n_samples)
    flow_rate = np.clip(flow_rate, 0, 100)
    
    # Historical average for comparison
    historical_avg_density = np.random.normal(2.0, 0.5, n_samples)
    historical_avg_traffic = np.random.normal(12, 3, n_samples)
    
    # Density ratio (current vs historical)
    density_ratio = base_density / (historical_avg_density + 0.1)
    traffic_ratio = traffic_flow / (historical_avg_traffic + 0.1)
    
    # Risk indicators
    congestion_level = np.where(
        (density_ratio > 2.5) & (flow_rate < 20), 1,  # High density, low flow
        np.where((density_ratio > 2.0) & (traffic_ratio > 1.5), 1, 0)  # High density and traffic
    )
    
    # Create target variable (disaster risk)
    # Risk factors: high density ratio, low velocity, special events, congestion
    risk_score = (
        (density_ratio - 1) * 0.3 +
        (1 / (crowd_velocity + 0.1)) * 0.2 +
        special_events * 0.3 +
        congestion_level * 0.2
    )
    
    # Add some noise and create binary classification
    risk_score += np.random.normal(0, 0.1, n_samples)
    disaster_risk = np.where(risk_score > 1.2, 1, 0)  # Binary classification
    
    # Create DataFrame
    data = pd.DataFrame({
        'timestamp': timestamps,
        'hour': hours,
        'day_of_week': days_of_week,
        'is_weekend': is_weekend,
        'zone': zones,
        'crowd_density': base_density,
        'traffic_flow': traffic_flow,
        'weather': weather_conditions,
        'temperature': temperature,
        'special_event': special_events,
        'crowd_velocity': crowd_velocity,
        'flow_rate': flow_rate,
        'historical_avg_density': historical_avg_density,
        'historical_avg_traffic': historical_avg_traffic,
        'density_ratio': density_ratio,
        'traffic_ratio': traffic_ratio,
        'congestion_level': congestion_level,
        'risk_score': risk_score,
        'disaster_risk': disaster_risk
    })
    
    return data

In [ ]:
# Generate synthetic data
print("📊 Generating synthetic crowd and traffic data...")
crowd_data = generate_synthetic_crowd_data(10000)
print(f"✅ Generated {len(crowd_data)} data points")
print(f"🚨 Disaster risk cases: {crowd_data['disaster_risk'].sum()} ({crowd_data['disaster_risk'].mean()*100:.1f}%)")

In [ ]:
## 2. EXPLORATORY DATA ANALYSIS

def analyze_crowd_patterns(data):
    """
    Comprehensive analysis of crowd patterns and risk factors
    """
    print("\n" + "="*60)
    print("📈 CROWD PATTERN ANALYSIS")
    print("="*60)
    
    # Basic statistics
    print("\n📊 Dataset Overview:")
    print(f"Total records: {len(data):,}")
    print(f"Date range: {data['timestamp'].min()} to {data['timestamp'].max()}")
    print(f"High-risk incidents: {data['disaster_risk'].sum():,} ({data['disaster_risk'].mean()*100:.2f}%)")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Crowd Disaster Risk Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Crowd density distribution by risk level
    axes[0,0].hist(data[data['disaster_risk']==0]['crowd_density'], alpha=0.7, label='Low Risk', bins=30, color='green')
    axes[0,0].hist(data[data['disaster_risk']==1]['crowd_density'], alpha=0.7, label='High Risk', bins=30, color='red')
    axes[0,0].set_xlabel('Crowd Density (people/m²)')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title('Crowd Density Distribution by Risk Level')
    axes[0,0].legend()
    
    # 2. Risk by hour of day
    hourly_risk = data.groupby('hour')['disaster_risk'].agg(['mean', 'count']).reset_index()
    axes[0,1].bar(hourly_risk['hour'], hourly_risk['mean'], color='orange', alpha=0.7)
    axes[0,1].set_xlabel('Hour of Day')
    axes[0,1].set_ylabel('Risk Probability')
    axes[0,1].set_title('Disaster Risk Probability by Hour')
    axes[0,1].set_xticks(range(0, 24, 2))
    
    # 3. Density ratio vs Flow rate (risk identification)
    scatter_colors = ['red' if risk else 'blue' for risk in data['disaster_risk']]
    axes[0,2].scatter(data['density_ratio'], data['flow_rate'], c=scatter_colors, alpha=0.6, s=20)
    axes[0,2].set_xlabel('Density Ratio (Current/Historical)')
    axes[0,2].set_ylabel('Flow Rate (people/min)')
    axes[0,2].set_title('Risk Pattern: Density vs Flow Rate')
    axes[0,2].axvline(x=2.0, color='red', linestyle='--', alpha=0.7, label='High Density Threshold')
    axes[0,2].axhline(y=30, color='red', linestyle='--', alpha=0.7, label='Low Flow Threshold')
    axes[0,2].legend()
    
    # 4. Zone-wise risk distribution
    zone_risk = data.groupby('zone')['disaster_risk'].agg(['mean', 'count']).reset_index()
    axes[1,0].bar(zone_risk['zone'], zone_risk['mean'], color='purple', alpha=0.7)
    axes[1,0].set_xlabel('Zone')
    axes[1,0].set_ylabel('Risk Probability')
    axes[1,0].set_title('Risk Probability by Zone')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # 5. Traffic flow impact on risk
    axes[1,1].boxplot([data[data['disaster_risk']==0]['traffic_flow'], 
                      data[data['disaster_risk']==1]['traffic_flow']], 
                     labels=['Low Risk', 'High Risk'])
    axes[1,1].set_ylabel('Traffic Flow (vehicles/min)')
    axes[1,1].set_title('Traffic Flow Distribution by Risk Level')
    
    # 6. Time series of risk incidents
    daily_risk = data.set_index('timestamp').resample('D')['disaster_risk'].sum()
    axes[1,2].plot(daily_risk.index, daily_risk.values, color='darkred', linewidth=2)
    axes[1,2].set_xlabel('Date')
    axes[1,2].set_ylabel('Daily Risk Incidents')
    axes[1,2].set_title('Risk Incidents Over Time')
    axes[1,2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return hourly_risk, zone_risk

# Perform analysis
hourly_patterns, zone_patterns = analyze_crowd_patterns(crowd_data)

In [ ]:
## 3. REAL-TIME ANOMALY DETECTION SYSTEM

class CrowdAnomalyDetector:
    """
    Real-time anomaly detection for unusual crowd patterns
    """
    
    def __init__(self):
        self.isolation_forest = IsolationForest(contamination=0.1, random_state=42)
        self.scaler = StandardScaler()
        self.is_fitted = False
        
    def fit(self, data):
        """
        Train the anomaly detection model
        """
        features = ['crowd_density', 'traffic_flow', 'flow_rate', 'density_ratio', 'traffic_ratio']
        X = data[features].fillna(0)
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X)
        
        # Fit isolation forest
        self.isolation_forest.fit(X_scaled)
        self.is_fitted = True
        
        print("✅ Anomaly detection model trained successfully")
        
    def detect_anomalies(self, data):
        """
        Detect anomalies in real-time data
        """
        if not self.is_fitted:
            raise ValueError("Model must be fitted first")
            
        features = ['crowd_density', 'traffic_flow', 'flow_rate', 'density_ratio', 'traffic_ratio']
        X = data[features].fillna(0)
        X_scaled = self.scaler.transform(X)
        
        # Predict anomalies (-1 = anomaly, 1 = normal)
        anomaly_predictions = self.isolation_forest.predict(X_scaled)
        anomaly_scores = self.isolation_forest.decision_function(X_scaled)
        
        return anomaly_predictions, anomaly_scores


In [ ]:
# Initialize and train anomaly detector
print("\n🔍 Training Anomaly Detection System...")
anomaly_detector = CrowdAnomalyDetector()
anomaly_detector.fit(crowd_data)

# Detect anomalies in the dataset
anomaly_preds, anomaly_scores = anomaly_detector.detect_anomalies(crowd_data)
crowd_data['anomaly'] = anomaly_preds
crowd_data['anomaly_score'] = anomaly_scores

print(f"🚨 Detected {(anomaly_preds == -1).sum()} anomalous patterns out of {len(crowd_data)} records")


In [ ]:
## 4. MACHINE LEARNING RISK PREDICTION MODEL

class DisasterRiskPredictor:
    """
    Advanced machine learning model for disaster risk prediction
    """
    
    def __init__(self):
        self.rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            class_weight='balanced'
        )
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.feature_importance = None
        
    def prepare_features(self, data, fit_encoders=True):
        """
        Prepare features for machine learning
        """
        # Select features
        numerical_features = [
            'hour', 'day_of_week', 'is_weekend', 'crowd_density', 'traffic_flow',
            'temperature', 'special_event', 'crowd_velocity', 'flow_rate',
            'density_ratio', 'traffic_ratio', 'congestion_level'
        ]
        
        categorical_features = ['zone', 'weather']
        
        # Prepare numerical features
        X_num = data[numerical_features].fillna(0)
        
        # Prepare categorical features
        X_cat = data[categorical_features].copy()
        
        for col in categorical_features:
            if fit_encoders:
                if col not in self.label_encoders:
                    self.label_encoders[col] = LabelEncoder()
                X_cat[col] = self.label_encoders[col].fit_transform(X_cat[col].fillna('Unknown'))
            else:
                if col in self.label_encoders:
                    # Handle unknown categories
                    known_categories = set(self.label_encoders[col].classes_)
                    X_cat[col] = X_cat[col].fillna('Unknown')
                    X_cat[col] = X_cat[col].apply(lambda x: x if x in known_categories else 'Unknown')
                    X_cat[col] = self.label_encoders[col].transform(X_cat[col])
                else:
                    X_cat[col] = 0
        
        # Combine features
        X = pd.concat([X_num, X_cat], axis=1)
        
        return X
    
    def train(self, data):
        """
        Train the disaster risk prediction model
        """
        print("\n🤖 Training Disaster Risk Prediction Model...")
        
        # Prepare features and target
        X = self.prepare_features(data, fit_encoders=True)
        y = data['disaster_risk']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        self.rf_model.fit(X_train_scaled, y_train)
        
        # Evaluate model
        train_score = self.rf_model.score(X_train_scaled, y_train)
        test_score = self.rf_model.score(X_test_scaled, y_test)
        
        print(f"✅ Model trained successfully")
        print(f"📈 Training accuracy: {train_score:.3f}")
        print(f"📈 Testing accuracy: {test_score:.3f}")
        
        # Get feature importance
        self.feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': self.rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Predictions for evaluation
        y_pred = self.rf_model.predict(X_test_scaled)
        
        print("\n📊 Classification Report:")
        print(classification_report(y_test, y_pred, 
                                  target_names=['Low Risk', 'High Risk']))
        
        return X_test, y_test, y_pred
    
    def predict_risk(self, data):
        """
        Predict disaster risk for new data
        """
        X = self.prepare_features(data, fit_encoders=False)
        X_scaled = self.scaler.transform(X)
        
        # Get predictions and probabilities
        predictions = self.rf_model.predict(X_scaled)
        probabilities = self.rf_model.predict_proba(X_scaled)
        
        return predictions, probabilities
    
    def plot_feature_importance(self):
        """
        Plot feature importance
        """
        plt.figure(figsize=(10, 8))
        top_features = self.feature_importance.head(10)
        
        plt.barh(range(len(top_features)), top_features['importance'], color='skyblue')
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Feature Importance')
        plt.title('Top 10 Most Important Features for Risk Prediction')
        plt.gca().invert_yaxis()
        
        for i, v in enumerate(top_features['importance']):
            plt.text(v + 0.001, i, f'{v:.3f}', va='center')
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Train the risk prediction model
risk_predictor = DisasterRiskPredictor()
X_test, y_test, y_pred = risk_predictor.train(crowd_data)

# Plot feature importance
risk_predictor.plot_feature_importance()


In [ ]:
## 5. REAL-TIME MONITORING AND ALERT SYSTEM

class RealTimeMonitor:
    """
    Real-time monitoring system for crowd disaster prevention
    """
    
    def __init__(self, risk_predictor, anomaly_detector):
        self.risk_predictor = risk_predictor
        self.anomaly_detector = anomaly_detector
        self.alert_thresholds = {
            'HIGH_RISK': 0.7,      # High disaster risk probability
            'ANOMALY': -0.3,       # Anomaly score threshold
            'DENSITY_RATIO': 2.5,   # Density compared to historical
            'LOW_FLOW': 20          # Low crowd flow rate
        }
        
    def analyze_current_situation(self, current_data):
        """
        Analyze current crowd situation and generate alerts
        """
        # Predict risk
        risk_pred, risk_prob = self.risk_predictor.predict_risk(current_data)
        
        # Detect anomalies
        anomaly_pred, anomaly_score = self.anomaly_detector.detect_anomalies(current_data)
        
        # Generate comprehensive assessment
        assessment = {
            'timestamp': current_data['timestamp'].iloc[0] if 'timestamp' in current_data else datetime.now(),
            'zone': current_data['zone'].iloc[0] if 'zone' in current_data else 'Unknown',
            'risk_probability': risk_prob[0][1] if len(risk_prob) > 0 else 0,
            'is_high_risk': risk_pred[0] == 1 if len(risk_pred) > 0 else False,
            'is_anomaly': anomaly_pred[0] == -1 if len(anomaly_pred) > 0 else False,
            'anomaly_score': anomaly_score[0] if len(anomaly_score) > 0 else 0,
            'crowd_density': current_data['crowd_density'].iloc[0],
            'density_ratio': current_data['density_ratio'].iloc[0],
            'flow_rate': current_data['flow_rate'].iloc[0],
            'traffic_flow': current_data['traffic_flow'].iloc[0],
        }
        
        # Generate alerts
        alerts = self.generate_alerts(assessment)
        assessment['alerts'] = alerts
        
        return assessment
    
    def generate_alerts(self, assessment):
        """
        Generate specific alerts based on assessment
        """
        alerts = []
        
        # High risk alert
        if assessment['risk_probability'] > self.alert_thresholds['HIGH_RISK']:
            alerts.append({
                'level': 'CRITICAL',
                'type': 'HIGH_DISASTER_RISK',
                'message': f"🚨 CRITICAL: High disaster risk detected! Probability: {assessment['risk_probability']:.1%}",
                'action': 'Implement crowd control measures immediately'
            })
        
        # Anomaly alert
        if assessment['anomaly_score'] < self.alert_thresholds['ANOMALY']:
            alerts.append({
                'level': 'WARNING',
                'type': 'ANOMALY_DETECTED',
                'message': f"⚠️  WARNING: Unusual crowd pattern detected (Score: {assessment['anomaly_score']:.2f})",
                'action': 'Monitor situation closely'
            })
        
        # High density alert
        if assessment['density_ratio'] > self.alert_thresholds['DENSITY_RATIO']:
            alerts.append({
                'level': 'WARNING',
                'type': 'HIGH_DENSITY',
                'message': f"⚠️  WARNING: Crowd density {assessment['density_ratio']:.1f}x higher than normal",
                'action': 'Consider crowd dispersal measures'
            })
        
        # Low flow alert (potential bottleneck)
        if (assessment['crowd_density'] > 3.0 and 
            assessment['flow_rate'] < self.alert_thresholds['LOW_FLOW']):
            alerts.append({
                'level': 'WARNING',
                'type': 'LOW_FLOW_BOTTLENECK',
                'message': f"⚠️  WARNING: Potential bottleneck - High density ({assessment['crowd_density']:.1f}) with low flow ({assessment['flow_rate']:.1f})",
                'action': 'Check for obstructions and improve flow'
            })
        
        return alerts
    
    def print_assessment(self, assessment):
        """
        Print formatted assessment report
        """
        print("\n" + "="*60)
        print("🔍 REAL-TIME CROWD SITUATION ASSESSMENT")
        print("="*60)
        print(f"📍 Zone: {assessment['zone']}")
        print(f"⏰ Time: {assessment['timestamp']}")
        print(f"🎯 Risk Probability: {assessment['risk_probability']:.1%}")
        print(f"👥 Crowd Density: {assessment['crowd_density']:.1f} people/m²")
        print(f"📊 Density Ratio: {assessment['density_ratio']:.1f}x historical")
        print(f"🌊 Flow Rate: {assessment['flow_rate']:.1f} people/min")
        print(f"🚗 Traffic Flow: {assessment['traffic_flow']:.1f} vehicles/min")
        
        if assessment['alerts']:
            print(f"\n🚨 ACTIVE ALERTS ({len(assessment['alerts'])})")
            print("-" * 40)
            for alert in assessment['alerts']:
                print(f"[{alert['level']}] {alert['message']}")
                print(f"   → Action: {alert['action']}")
                print()
        else:
            print("\n✅ No alerts - Situation appears normal")



In [ ]:
# Initialize real-time monitor
print("\n📡 Initializing Real-Time Monitoring System...")
real_time_monitor = RealTimeMonitor(risk_predictor, anomaly_detector)
print("✅ Real-time monitoring system ready")

In [ ]:
## 6. SIMULATION OF REAL-TIME MONITORING

def simulate_real_time_monitoring(data, monitor, n_samples=5):
    """
    Simulate real-time monitoring with sample data points
    """
    print("\n" + "="*60)
    print("🎬 SIMULATING REAL-TIME MONITORING")
    print("="*60)
    
    # Sample some high-risk and normal situations
    high_risk_samples = data[data['disaster_risk'] == 1].sample(min(3, len(data[data['disaster_risk'] == 1])))
    normal_samples = data[data['disaster_risk'] == 0].sample(min(2, len(data[data['disaster_risk'] == 0])))
    
    sample_data = pd.concat([high_risk_samples, normal_samples]).sample(n_samples)
    
    for idx, (_, row) in enumerate(sample_data.iterrows(), 1):
        print(f"\n📊 MONITORING SAMPLE {idx}/{n_samples}")
        
        # Convert row to DataFrame for processing
        current_data = pd.DataFrame([row])
        
        # Analyze situation
        assessment = monitor.analyze_current_situation(current_data)
        
        # Print assessment
        monitor.print_assessment(assessment)
        
        print("-" * 60)

In [ ]:
# Run simulation
simulate_real_time_monitoring(crowd_data, real_time_monitor, n_samples=3)

In [ ]:
## 7. RECOMMENDATIONS AND ACTION PLANS

def generate_prevention_recommendations():
    """
    Generate comprehensive disaster prevention recommendations
    """
    recommendations = {
        'immediate_actions': [
            "Deploy additional security personnel in high-risk zones",
            "Set up crowd flow monitoring cameras at bottleneck points",
            "Establish emergency evacuation routes and mark them clearly",
            "Install crowd density sensors in critical areas",
            "Create real-time communication system with field teams"
        ],
        
        'crowd_management': [
            "Implement one-way flow systems in narrow passages",
            "Use barriers to guide crowd movement and prevent clustering",
            "Deploy crowd control personnel at strategic chokepoints",
            "Create multiple entry/exit points to distribute flow",
            "Use public announcement systems for crowd direction"
        ],
        
        'technology_integration': [
            "Install AI-powered video analytics for automatic crowd counting",
            "Deploy IoT sensors for real-time density measurement",
            "Use mobile apps to provide crowd updates to visitors",
            "Implement predictive modeling for event planning",
            "Set up automated alert systems for emergency response"
        ],
        
        'emergency_response': [
            "Train staff on stampede prevention and response protocols",
            "Establish rapid response teams at strategic locations",
            "Create communication protocols with local emergency services",
            "Develop evacuation procedures for different scenarios",
            "Conduct regular emergency drills and simulations"
        ]
    }
    
    print("\n" + "="*60)
    print("📋 COMPREHENSIVE PREVENTION RECOMMENDATIONS")
    print("="*60)
    
    for category, actions in recommendations.items():
        print(f"\n🎯 {category.upper().replace('_', ' ')}")
        print("-" * 40)
        for i, action in enumerate(actions, 1):
            print(f"{i}. {action}")
    
    return recommendations

In [ ]:
# Generate recommendations
prevention_recommendations = generate_prevention_recommendations()

In [ ]:
## 8. MODEL PERFORMANCE DASHBOARD

def create_performance_dashboard(data, y_test, y_pred):
    """
    Create comprehensive model performance dashboard
    """
    print("\n" + "="*60)
    print("📊 MODEL PERFORMANCE DASHBOARD")
    print("="*60)
    
    # Create confusion matrix visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Crowd Disaster Prediction System - Performance Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
                xticklabels=['Low Risk', 'High Risk'],
                yticklabels=['Low Risk', 'High Risk'])
    axes[0,0].set_title('Confusion Matrix')
    axes[0,0].set_ylabel('Actual')
    axes[0,0].set_xlabel('Predicted')
    
    # 2. Risk Distribution
    risk_dist = data['disaster_risk'].value_counts()
    axes[0,1].pie(risk_dist.values, labels=['Low Risk', 'High Risk'], 
                  autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    axes[0,1].set_title('Risk Distribution in Dataset')
    
    # 3. Anomaly Detection Results
    anomaly_dist = pd.Series(data['anomaly']).value_counts()
    axes[1,0].bar(['Normal', 'Anomaly'], [anomaly_dist[1], anomaly_dist[-1]], 
                  color=['blue', 'red'], alpha=0.7)
    axes[1,0].set_title('Anomaly Detection Results')
    axes[1,0].set_ylabel('Count')
    
    # 4. Risk Score Distribution
    axes[1,1].hist(data[data['disaster_risk']==0]['risk_score'], alpha=0.7, 
                   label='Low Risk', bins=30, color='green')
    axes[1,1].hist(data[data['disaster_risk']==1]['risk_score'], alpha=0.7, 
                   label='High Risk', bins=30, color='red')
    axes[1,1].set_xlabel('Risk Score')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].set_title('Risk Score Distribution')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
 # Print key metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📈 MODEL PERFORMANCE METRICS:")
print(f"   Accuracy:  {accuracy:.3f}")
print(f"   Precision: {precision:.3f}")
print(f"   Recall:    {recall:.3f}")
print(f"   F1-Score:  {f1:.3f}")